In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# %qtconsole

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
			
import pandas as pd
import numpy as np
from pathlib import Path		

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [4]:
df = pd.read_csv(r'data/merged_date_v2.csv', parse_dates = ['count_date'], index_col = 'count_date')
df.head()

,road_name,adt,latitude,longitude,sampling_count
count_date,,,,,
1974-07-12,COLWILL RD,125.0,-36.832191,174.636777,3
1974-08-11,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22
1974-08-11,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1
1974-08-11,TAIKATA RD,2490.0,-36.841192,174.651345,1
1974-08-11,GLORIA AVE,1823.0,-36.848881,174.650064,5


In [ ]:
# add date parts


In [ ]:
# train test split
train, test = 